In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold #Something to split and randomise the data 

from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [6]:
data = pd.read_csv('Credit/creditdefault_train.csv') #Putting the test data and the train data in data frames 
test = pd.read_csv('Credit/creditdefault_test.csv')

#Seperating the train data labels from the inputs:
y_train = data['Y']
x_train = data.drop(['Y'],axis=1)

#Seperating the test data labels from the inputs:
y_test = test['Y']
x_test = test.drop(['Y'],axis=1)

kf = StratifiedKFold(n_splits=4)
#kf = KFold(15000, n_folds=3)



In [13]:
random_forest = ensemble.RandomForestClassifier(n_estimators=500)

In [9]:
random_forest.fit(x_train,y_train)

RandomForestClassifier(n_estimators=500)

In [11]:
y_pred = random_forest.predict(x_test)

In [14]:
print(accuracy_score(y_test,y_pred))

0.8168


In [28]:
random_forest = ensemble.RandomForestClassifier(n_estimators=750)

In [29]:
random_forest.fit(x_train,y_train)

RandomForestClassifier(n_estimators=750)

In [26]:
y_pred = random_forest.predict(x_test)

In [27]:
print(accuracy_score(y_test,y_pred))

0.8152666666666667


Whats consuming the most time is the fitting of the data.